In [2]:
%%time

import os
import re
import string
import pprint

import pandas as pd
import numpy as np

import gensim

Wall time: 0 ns


In [3]:
%%time

# change working directory

print('Before chdir:', os.getcwd())

os.chdir('F:\\CG\\w2vCustom') 
         
print('After chdir:', os.getcwd())


Before chdir: C:\Users\Ravi
After chdir: F:\CG\w2vCustom
Wall time: 1 ms


In [4]:
%%time

df=pd.read_excel("wb_causecode_asgrp.xlsx");

df.head()


Wall time: 23.5 s


,account,app_name,assigned_group,causecode_lv1,causecode_lv2,causecode_lv3,incident_id,summary
0,wb,NaN,WBEI.Sharepoint,Access,Issue,Unable to access application,RTASK0033182,User cannot access Time-Off Tracker
1,wb,SAP Accounts Payable/Workflow,WBEI.MKTMGT.Siebel North America,Delete,Data Deletion,As per user requirement,RTASK0086491,"Promo1169410 ACB#236740 Deal Status ""Payment i..."
2,wb,NaN,WBEI.MIS.Alfred,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0106734,Error messge stating that the Alternate Payee ...
3,wb,SAP Netweaver Business Warehouse,WBEI.BW - Data Warehouse,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0246906,FRB Reconciliation need to be perform on 06/04...
4,wb,System,WBEI.WBCAS Adobe LiveCycle,Access,Approval,Access/SAP Issue/Unable to access environment,RTASK0286950,!! URGENT-- Forward eform 11828 to another app...


In [5]:
%%time

# Check missing values in a dataframe
# df.isnull().values.any()

Wall time: 0 ns


In [6]:
%%time

# Function to keep required columns in a dataframe

def keepcols(df, cols):
    '''
    df = dataframe
    cols = list of columns to keep in df 
    '''
    
    cols_keep = [cols[0], cols[1], cols[2], cols[3]]
    return df[cols_keep]

Wall time: 0 ns


In [8]:
%%time

# Subset required columns
df1 = keepcols(df, ['causecode_lv1', 'causecode_lv2', 'causecode_lv3', 'summary'])

df1.head()

Wall time: 16 ms


,causecode_lv1,causecode_lv2,causecode_lv3,summary
0,Access,Issue,Unable to access application,User cannot access Time-Off Tracker
1,Delete,Data Deletion,As per user requirement,"Promo1169410 ACB#236740 Deal Status ""Payment i..."
2,Access,Approval,Access/SAP Issue/Unable to access environment,Error messge stating that the Alternate Payee ...
3,Access,Approval,Access/SAP Issue/Unable to access environment,FRB Reconciliation need to be perform on 06/04...
4,Access,Approval,Access/SAP Issue/Unable to access environment,!! URGENT-- Forward eform 11828 to another app...


In [9]:
%%time

# Check for missing vlaues after iputation in a dataframe

print("Before", df1.isnull().values.any())


# check and impute missing values in a dataframe


df1 = df1.fillna('')
df2 = df1.head(5000)

print("After", df1.isnull().values.any())

Before True
After False
Wall time: 55 ms


In [10]:
%%time

def combined_features(df):
    combi_feats = []
    for i in range(0, df.shape[0]):
        combi_feat = df['causecode_lv1'][i]+' '+df['causecode_lv2'][i]+' '+df['causecode_lv3'][i]+' '+df['summary'][i]
        combi_feats.append(combi_feat)
    
    return combi_feats




#          combi_feat = df['causecode_lv1'][i]+' '+df['causecode_lv2'][i]+' '+df['causecode_lv3'][i]+' '+df['summary'][i]  
#         combi_feat = df[lstcols[0]][i]+' '+df[lstcols[1]][i]+' '+df[lstcols[2]][i]+' '+df[lstcols[3]][i]
#         combi_feat = df.loc[i,lstcols[0]]+' '+df[i, lstcols[1]]+' '+df[lstcols[2]]+' '+df[i, lstcols[3]]
     

Wall time: 0 ns


In [11]:
%%time

df2['combined_feats'] = combined_features(df2)
df2

Wall time: 169 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,causecode_lv1,causecode_lv2,causecode_lv3,summary,combined_feats
0,Access,Issue,Unable to access application,User cannot access Time-Off Tracker,Access Issue Unable to access application User...
1,Delete,Data Deletion,As per user requirement,"Promo1169410 ACB#236740 Deal Status ""Payment i...",Delete Data Deletion As per user requirement P...
2,Access,Approval,Access/SAP Issue/Unable to access environment,Error messge stating that the Alternate Payee ...,Access Approval Access/SAP Issue/Unable to ac...
3,Access,Approval,Access/SAP Issue/Unable to access environment,FRB Reconciliation need to be perform on 06/04...,Access Approval Access/SAP Issue/Unable to ac...
4,Access,Approval,Access/SAP Issue/Unable to access environment,!! URGENT-- Forward eform 11828 to another app...,Access Approval Access/SAP Issue/Unable to ac...
...,...,...,...,...,...
4995,Access,Issue,Unable to access application,TOBI_Request for BO user account creation- XKB...,Access Issue Unable to access application TOBI...
4996,Access,Request,Remove Access,Leavesden Leavers - 9/29 & 10/1/2017,Access Request Remove Access Leavesden Leavers...
4997,Access,Request,New Access,update XSBOMMAR in WPP to assign WZ:605 and re...,Access Request New Access update XSBOMMAR in W...
4998,Access,Issue,Unable to access environment,[URGENT] Office 2016 migration,Access Issue Unable to access environment [UR...


In [58]:
%%time

import re
from gensim.parsing.preprocessing import remove_stopwords

# Preprocess text for summary
def clean_sentence(sentence, stopwords=False):
    
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #lower case 
    sentence = sentence.lower()
    # square brackets
    sentence = re.sub('\[.*?\]', '', sentence)  
    # hyperlinks
    sentence = re.sub('https?://\S+|www\.\S+', '', sentence)
    sentence = re.sub('<.*?>+', '', sentence)
    # remove punctuation
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)
    sentence = re.sub('\n', '', sentence)
    # remove year
    sentence = re.sub(r"\b(19[40][0-9]|20[0-1][0-9]|2020)\b",'',sentence)
    # remove month names
    mp = r"(\b\d{1,2}\D{0,3})?\b(?:Jan|jan(?:uary)?|Feb|feb(?:ruary)?|Mar|mar(?:ch)?|Apr|mar(?:il)?|May|may|Jun|jun(?:e)?|Jul|jul(?:y)?|Aug|aug(?:ust)?|Sep|sep(?:tember)?|Oct|oct(?:ober)?|(Nov|nov|Dec|dec)(?:ember)?)\D?(\d{1,2}(st|nd|rd|th)?)?(([,.\-\/])\D?)?((19[7-9]\d|20\d{2})|\d{2})*"
    sentence = re.sub(mp, '', sentence)
     
    # remove words containing numbers
    sentence = re.sub('\w*\d\w*', '', sentence)
    # remove numbers 
    sentence = re.sub(r'[0-9]+','',sentence)
    
    if stopwords:
        sentence = remove_stopwords(sentence)
    
    return sentence

Wall time: 0 ns


In [32]:
%%time

def get_cleaned_sentences(df, col, stopwords=True):
    cleaned_sents = []
    
    for index, row in df.iterrows():
        cleaned = clean_sentence(row[col])
        cleaned_sents.append(cleaned)
    return cleaned_sents


# Calling the function get_cleaned_text

cleaned_sentences = get_cleaned_sentences(df2, 'combined_feats')
cleaned_sentences

Wall time: 835 ms


['access issue unable to access application user cannot access timeoff tracker',
 'delete data deletion as per user requirement   deal status payment in progress with no payment',
 'access approval  accesssap issueunable to access environment error messge stating that the alternate payee group does not exist',
 'access approval  accesssap issueunable to access environment frb reconciliation need to be perform on  ',
 'access approval  accesssap issueunable to access environment  urgent forward eform  to another approver',
 'request enquiry information report migration from dev to qa',
 'access request unlock disable ei batch jobs',
 'access request unlock complete request  general service request',
 'access request remove access hpqc access to project workday domain mis  id xvdhake',
 'access issue unable to access environment  help withsetting up cbs profile in wbconnect',
 'access request unlock route ticket to new approver',
 'access request unlock proactive breakfix  incident ticke

In [46]:
%%time
sentences = cleaned_sentences
sentence_words = [[word for word in sent.split()]
                  for sent in sentences]

filtered_words = [[w for w in lst if len(w) > 2] for lst in sentence_words]         
                  
print(filtered_words)

                  
# [[w for w in lst if len(w) > 2] for lst in wordsx]                  
# print [[item for item in clist if len(item) >= 3] for clist in lists]

[['access', 'issue', 'unable', 'access', 'application', 'user', 'cannot', 'access', 'timeoff', 'tracker'], ['delete', 'data', 'deletion', 'per', 'user', 'requirement', 'deal', 'status', 'payment', 'progress', 'with', 'payment'], ['access', 'approval', 'accesssap', 'issueunable', 'access', 'environment', 'error', 'messge', 'stating', 'that', 'the', 'alternate', 'payee', 'group', 'does', 'not', 'exist'], ['access', 'approval', 'accesssap', 'issueunable', 'access', 'environment', 'frb', 'reconciliation', 'need', 'perform'], ['access', 'approval', 'accesssap', 'issueunable', 'access', 'environment', 'urgent', 'forward', 'eform', 'another', 'approver'], ['request', 'enquiry', 'information', 'report', 'migration', 'from', 'dev'], ['access', 'request', 'unlock', 'disable', 'batch', 'jobs'], ['access', 'request', 'unlock', 'complete', 'request', 'general', 'service', 'request'], ['access', 'request', 'remove', 'access', 'hpqc', 'access', 'project', 'workday', 'domain', 'mis', 'xvdhake'], ['acc

In [34]:
%%time
# Creating gensim word2vec model
import multiprocessing
from gensim.models import Word2Vec

# Count no. of cores in computer
cores = multiprocessing.cpu_count()

# Model parameters
wb_word2vec_model = Word2Vec(min_count=5,
                            window=5,
                            size=300,
                            sample=6e-5,
                            alpha=0.03,
                            min_alpha=0.0007,
                            negative=20,
                            workers=cores-1)


print(wb_word2vec_model)

Word2Vec(vocab=0, size=300, alpha=0.03)
Wall time: 998 µs


In [35]:
%%time
# Build vacoubalry
wb_word2vec_model.build_vocab(filtered_words)


print(wb_word2vec_model)



Word2Vec(vocab=773, size=300, alpha=0.03)
Wall time: 243 ms


In [36]:
%%time
# Training the model
wb_word2vec_model.train(filtered_words, total_examples=wb_word2vec_model.corpus_count, epochs=30)


Wall time: 1.15 s


(263123, 1586940)

In [37]:
# summarize vocabulary
words = list(wb_word2vec_model.wv.vocab)
print(words)

['access', 'issue', 'unable', 'application', 'user', 'cannot', 'delete', 'data', 'deletion', 'per', 'requirement', 'deal', 'status', 'payment', 'with', 'approval', 'accesssap', 'issueunable', 'environment', 'error', 'that', 'the', 'group', 'does', 'not', 'frb', 'reconciliation', 'need', 'perform', 'urgent', 'eform', 'approver', 'request', 'enquiry', 'information', 'report', 'migration', 'from', 'dev', 'unlock', 'batch', 'jobs', 'complete', 'general', 'service', 'remove', 'project', 'workday', 'domain', 'help', 'cbs', 'profile', 'wbconnect', 'ticket', 'new', 'incident', 'for', 'monthly', 'release', 'into', 'production', 'add', 'hierarchy', 'quantity', 'change', 'order', 'configure', 'refresh', 'out', 'emergency', 'email', 'job', 'failure', 'list', 'have', 'role', 'company', 'code', 'please', 'migrate', 'objects', 'shopping', 'cart', 'master', 'srm', 'issues', 'update', 'modify', 'support', 'when', 'log', 'adobe', 'attention', 'required', 'form', 'applicant', 'invalid', 'found', 'sap', '

In [38]:
%%time
# Save word2vec model
wb_word2vec_model.save("./wb_word2vec_model.model")

# Save WV
wb_word2vec_model.wv.save_word2vec_format("wb_word2vec_model.txt")

Wall time: 224 ms


In [39]:
%%time
# Load the custom wb_word2vec_model 

wb_word2vec_model = gensim.models.KeyedVectors.load_word2vec_format("./wb_word2vec_model.txt", binary = False)
print("wb_word2vec_model")

wb_word2vec_model
Wall time: 346 ms


In [40]:
%%time
# Create BOW
# Create corpora
from gensim import corpora


dictionary = corpora.Dictionary(filtered_words)

# Creating BOW

bow_corpus = [dictionary.doc2bow(word) for word in filtered_words]
for sent,embedding in zip(cleaned_sentences,bow_corpus):
    print(sent)
    print(embedding)

access issue unable to access application user cannot access timeoff tracker
[(0, 3), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
delete data deletion as per user requirement   deal status payment in progress with no payment
[(7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]
access approval  accesssap issueunable to access environment error messge stating that the alternate payee group does not exist
[(0, 2), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1)]
access approval  accesssap issueunable to access environment frb reconciliation need to be perform on  
[(0, 2), (18, 1), (20, 1), (22, 1), (26, 1), (33, 1), (34, 1), (35, 1), (36, 1)]
access approval  accesssap issueunable to access environment  urgent forward eform  to another approver
[(0, 2), (18, 1), (20, 1), (22, 1), (26, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1)]
request enquiry

In [77]:
%%time
#question_orig="do I need to learn algorithms to be a data scientist ?";
query= "Unable to access application"
query=clean_sentence(query,stopwords=True);
query_embedding = dictionary.doc2bow(query.split())


print("\n\n",query,"\n",query_embedding)



 unable access application 
 [(0, 1), (1, 1), (6, 1)]
Wall time: 1 ms


In [78]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def get_cosim(query,docs):
      
    count = []
    simscore = []
    result = []

    for index,faq_embedding in enumerate(docs):
            sim=cosine_similarity(faq_embedding,query)[0][0]

            count.append(index)
            simscore.append(sim)
            result.append(sentences[index])



    dfScore = pd.DataFrame(list(zip(count, simscore, result)),
                  columns=['id','simscore', 'result'])


    dfsort = dfScore.sort_values('simscore', ascending = False)


    return dfsort


In [79]:
# Getting Phrase Embeddings from Word Embeddings

def getWordVec(word,model):
        samp=model['users']
        vec=[0]*len(samp);
        try:
                vec=model[word]
        except:
                vec=[0]*len(samp)
        return (vec)


def getPhraseEmbedding(phrase,embeddingmodel):
                       
        samp=getWordVec('users', embeddingmodel)
        vec=np.array([0]*len(samp));
        den=0;
        for word in phrase.split():
            #print(word)
            den=den+1;
            vec=vec+np.array(getWordVec(word,embeddingmodel))
        #vec=vec/den;
        #return (vec.tolist());
        return vec.reshape(1, -1)

In [80]:
%%time
#With wb_word2vec_model



sent_embeddings=[]
for sent in sentences:
    sent_embeddings.append(getPhraseEmbedding(sent,wb_word2vec_model))

question_embedding=getPhraseEmbedding(query,wb_word2vec_model)

dff = get_cosim(question_embedding,sent_embeddings).head(5)

dff

Wall time: 2.84 s


,id,simscore,result
2332,2332,0.999760,access issue unable to access application unab...
4680,4680,0.999747,access issue unable to access application sear...
3905,3905,0.999664,access issue unable to access application enab...
4476,4476,0.999655,access issue unable to access application epm...
1241,1241,0.999645,access issue unable to access application qa a...


In [94]:
for i in dff.index:
    print(i)
    print(dff['result'][i])
    
    

2332
access issue unable to access application unable to print  extreme slowness
4680
access issue unable to access application search results inquiry
3905
access issue unable to access application enable workday integration  sap gl in slhub qa
4476
access issue unable to access application  epm plugin  install
1241
access issue unable to access application qa approval  
